In [31]:
import os
import numpy as np
import pandas as pd
from glob import glob
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import re
import shutil
from sklearn import preprocessing

In [ ]:
# BreaKHis_v1/histology_slides/breast/benign/SOB/adenosis/SOB_B_A_14-22549AB/40X/SOB_B_A-14-22549AB-40-001.png
benignfiles=glob('./BreaKHis_v1/histology_slides/breast/benign/SOB/*/*/*/*.png')
malignantfiles =glob('./BreaKHis_v1/histology_slides/breast/malignant/SOB/*/*/*/*.png')
fileslist = benignfiles+malignantfiles

print(len(benignfiles))
print(len(malignantfiles))
print(len(fileslist))

In [ ]:
def extract_info(filelist):
    filenames=[]
    patient = []
    cat=[]
    subcat=[]
    mag =[]
    for file in tqdm(fileslist):
        curl = file.split('/')
        spt=curl[-3].split('_')
        pt =spt[-1]
        mag.append(curl[-2])
        filenames.append(curl[-1])
        patient.append(pt)
        cat.append(spt[1])
        subcat.append(spt[2])
        
    return filenames, patient, cat, subcat, mag

In [ ]:
fnames, patient, cat, subcat, mag= extract_info(fileslist)

In [ ]:
df = pd.DataFrame(fileslist, columns=['filepath'])
df['filename']=fnames
df['Mag']=mag
df['patient_id']=patient
df['Tumor_type']=subcat
df['Tumor_class']=cat

In [ ]:
le = preprocessing.LabelEncoder()
rf =le.fit_transform(df['Tumor_type'].map(str))

cls =le.fit_transform(df['Tumor_class'].map(str))

df['Types']=rf
df['Classes']=cls

In [ ]:
df.head(3)

In [ ]:
df.to_csv('all_datafiles.csv', index=False)

# create training files and testing files.

In [ ]:
df =pd.read_csv('all_datafiles.csv')

In [ ]:
df.Mag.unique()

In [ ]:
tf=df[df.Mag=='400X']

In [ ]:
# Split data into training and testing sets
# maybe i should have done this using the whole data
train_df,test_df = train_test_split(tf, test_size = 0.3, random_state = 42)

In [ ]:
print(train_df.shape)
print(test_df.shape)

In [ ]:
train_df.to_csv('400xtrain_df.csv', index=False)
test_df.to_csv('400xtest_df.csv', index=False)

In [43]:
# # Array data is needed. not 
# def imagefeature(df):
#     path_list =df['filepath'].tolist()
#     height =[]
#     width =[]
#     feature =[]
#     types =[]
#     classes =[]
#     dim = (460, 700)
#     for src in tqdm(path_list):
#         fname = df[df.filepath==src]['filename'].values[0]
#         label = df[df.filepath==src]['Classes'].values[0]
#         lab = df[df.filepath==src]['Types'].values[0]
#         types.append(lab)
#         classes.append(label)
# #         shutil.copy2(src, dst)

#         image = cv2.imread(src)
#         image =  cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        

#         resized = cv2.resize(image, dim, interpolation = cv2.INTER_CUBIC)
#         feature.append(resized)
# #         height.append(image.shape[0])
# #         width.append(image.shape[1])
# #     df['height']=height
# #     df['width']=width
#     feature =np.array(feature)
#     classes =np.array(classes)
#     types =np.array(types)
#     return feature, classes, types

In [44]:
# mg4path ='40xtrain_df.csv'
# mg10path ='100xtrain_df.csv'
# mg20path ='200xtrain_df.csv'
# mg4path ='400xtrain_df.csv'


# tmg4path ='40xtest_df.csv'
# tmg10path ='100xtest_df.csv'
# tmg20path ='200xtest_df.csv'
# tmg4path ='400xtest_df.csv'

In [45]:
# train_df =pd.read_csv('100xtrain_df.csv')
# test_df =pd.read_csv('100xtest_df.csv')

In [46]:
# train_df.head(2)

filepath  \
0  ./breast/malignant/SOB/papillary_carcinoma/SOB...   
1  ./breast/malignant/SOB/mucinous_carcinoma/SOB_...   

                        filename   Mag patient_id Tumor_type Tumor_class  \
0  SOB_M_PC-14-19440-100-038.png  100X   14-19440         PC           M   
1  SOB_M_MC-14-12773-100-008.png  100X   14-12773         MC           M   

   Types  Classes  
0      5        1  
1      4        1

In [59]:

trainfeature, trainclasses, traintypes = imagefeature(train_df)
testfeature, testclasses, testtypes = imagefeature(test_df)
trainfeature.shape

100%|██████████| 625/625 [00:05<00:00, 121.86it/s]


(1456, 700, 460, 3)

In [60]:
# trainfeature =np.transpose(trainfeature, (0,2,1,3))

In [61]:
testfeature =np.transpose(testfeature, (0,2,1,3))
testfeature.shape

(625, 460, 700, 3)

In [62]:
trainclabels =np.asarray(pd.get_dummies(trainclasses))
trainslabels = np.asarray(pd.get_dummies(traintypes))
trainlabels = np.concatenate((trainclabels, trainslabels), axis=1)

In [63]:
testclabels =np.asarray(pd.get_dummies(testclasses))
testslabels = np.asarray(pd.get_dummies(testtypes))
testlabels = np.concatenate((testclabels, testslabels), axis=1)

In [66]:
np.save('traintype100X', traintypes)
np.save('testtype100X', testtypes)
np.save('trainclasses100X', trainclasses)
np.save('testclasses100X', testclasses)

np.save('feature100X', trainfeature)
np.save('testfeature100X', testfeature)
np.save('trainlabels100X', trainlabels)
np.save('testlabels100X', testlabels)

In [6]:
def create_folders(base_dir='data_40X'):
    # Create a new directory
    if not os.path.exists(base_dir):
        os.mkdir(base_dir)
    # now we create 2 folders inside 'root_dir':
    # train
        # benign
        # malignant

    # test
        # benign
        # malingnant

    train_dir = os.path.join(base_dir, 'train')
    if not os.path.exists(train_dir):
        os.mkdir(train_dir)

    # test_dir
    test_dir = os.path.join(base_dir, 'test')
    if not os.path.exists(test_dir):
        os.mkdir(test_dir)
    # create new folders inside train_dir

    benign = os.path.join(train_dir, 'B')
    if not os.path.exists(benign):
        os.mkdir(benign)
    malingnant = os.path.join(train_dir, 'M')
    if not os.path.exists(malingnant):
        os.mkdir(malingnant)

    # create new folders inside test_dir
    testbenign = os.path.join(test_dir, 'B')
    if not os.path.exists(testbenign):
        os.mkdir(testbenign)
    testmalingnant = os.path.join(test_dir, 'M')
    if not os.path.exists(testmalingnant):
        os.mkdir(testmalingnant)
    return train_dir, test_dir

In [7]:
# Transfer the train images
def Filetransfer(df, root_dir):
    path_list =df['filepath'].tolist()
    for src in tqdm(path_list):
        fname = df[df.filepath==src]['filename'].values[0]
        label = df[df.filepath==src]['Tumor_class'].values[0]
        dst = os.path.join(root_dir, label, fname)
        shutil.copy2(src, dst)

#         image = cv2.imread(src)
#         image = cv2.resize(image, (im_height, im_width))
#         # save the image at the destination
#         cv2.imwrite(dst, image)
   

In [15]:
mg4path ='40xtrain_df.csv'
mg10path ='100xtrain_df.csv'
mg20path ='200xtrain_df.csv'
mg40path ='400xtrain_df.csv'


tmg4path ='40xtest_df.csv'
tmg10path ='100xtest_df.csv'
tmg20path ='200xtest_df.csv'
tmg40path ='400xtest_df.csv'

In [16]:
# train_list =[mg4path,mg10path,mg20path, mg4path]


In [18]:
df =pd.read_csv(mg4path)
tf = pd.read_csv(tmg4path)
train_dir, test_dir =create_folders(base_dir='data_40X')
Filetransfer(df, train_dir)
Filetransfer(tf, test_dir)

100%|██████████| 599/599 [00:10<00:00, 58.87it/s]


In [24]:
def create_folders2(base_dir='data_40X'):
    # Create a new directory
    if not os.path.exists(base_dir):
        os.mkdir(base_dir)
    # now we create 2 folders inside 'root_dir':
    # train
        # benign
        # malignant
# array(['PT', 'PC', 'MC', 'DC', 'F', 'A', 'LC', 'TA']
    # test
        # benign
        # malingnant

    train_dir = os.path.join(base_dir, 'Mtrain')
    if not os.path.exists(train_dir):
        os.mkdir(train_dir)

    # test_dir
    test_dir = os.path.join(base_dir, 'Mtest')
    if not os.path.exists(test_dir):
        os.mkdir(test_dir)
    # create new folders inside train_dir

    benign = os.path.join(train_dir, 'A')
    if not os.path.exists(benign):
        os.mkdir(benign)
    benign  = os.path.join(train_dir, 'F')
    if not os.path.exists(benign):
        os.mkdir(benign)
    
    malingnant = os.path.join(train_dir, 'DC')
    if not os.path.exists(malingnant):
        os.mkdir(malingnant)
    malingnant = os.path.join(train_dir, 'MC')
    if not os.path.exists(malingnant):
        os.mkdir(malingnant)
      
    
    malingnant = os.path.join(train_dir, 'PC')
    if not os.path.exists(malingnant):
        os.mkdir(malingnant)
    benign = os.path.join(train_dir, 'PT')
    if not os.path.exists(benign):
        os.mkdir(benign)
    
      
    malingnant = os.path.join(train_dir, 'LC')
    if not os.path.exists(malingnant):
        os.mkdir(malingnant)
    benign = os.path.join(train_dir, 'TA')
    if not os.path.exists(benign):
        os.mkdir(benign)
    

    benign = os.path.join(test_dir, 'A')
    if not os.path.exists(benign):
        os.mkdir(benign)
    benign  = os.path.join(test_dir, 'F')
    if not os.path.exists(benign):
        os.mkdir(benign)
    
    malingnant = os.path.join(test_dir, 'DC')
    if not os.path.exists(malingnant):
        os.mkdir(malingnant)
    malingnant = os.path.join(test_dir, 'MC')
    if not os.path.exists(malingnant):
        os.mkdir(malingnant)
      
    
    malingnant = os.path.join(test_dir, 'PC')
    if not os.path.exists(malingnant):
        os.mkdir(malingnant)
    benign = os.path.join(test_dir, 'PT')
    if not os.path.exists(benign):
        os.mkdir(benign)
    
      
    malingnant = os.path.join(test_dir, 'LC')
    if not os.path.exists(malingnant):
        os.mkdir(malingnant)
    benign = os.path.join(test_dir, 'TA')
    if not os.path.exists(benign):
        os.mkdir(benign)
    return train_dir, test_dir

In [25]:
# Transfer the train images
def Filetransfer2(df, root_dir):
    path_list =df['filepath'].tolist()
    for src in tqdm(path_list):
        fname = df[df.filepath==src]['filename'].values[0]
        label = df[df.filepath==src]['Tumor_type'].values[0]
        dst = os.path.join(root_dir, label, fname)
        shutil.copy2(src, dst)

#         image = cv2.imread(src)
#         image = cv2.resize(image, (im_height, im_width))
#         # save the image at the destination
#         cv2.imwrite(dst, image)
        #shutil.copyfile(src, dst)

In [26]:
mg4path ='40xtrain_df.csv'
mg10path ='100xtrain_df.csv'
mg20path ='200xtrain_df.csv'
mg40path ='400xtrain_df.csv'


tmg4path ='40xtest_df.csv'
tmg10path ='100xtest_df.csv'
tmg20path ='200xtest_df.csv'
tmg40path ='400xtest_df.csv'

In [30]:
df =pd.read_csv(mg40path)
tf = pd.read_csv(tmg40path)
train_dir, test_dir =create_folders2(base_dir='data_400X')
Filetransfer2(df, train_dir)
Filetransfer2(tf, test_dir)

100%|██████████| 546/546 [00:00<00:00, 1190.92it/s]
